In [13]:
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter
import re
from pptx import Presentation
from pptx.util import Inches

# Path to Tesseract OCR binary (adjust the path for your system)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Function to preprocess the image for better OCR
def preprocess_image(image_path):
    image = Image.open(image_path)
    image = image.convert("L")  # Convert to grayscale
    image = ImageEnhance.Contrast(image).enhance(2)  # Increase contrast
    image = image.filter(ImageFilter.SHARPEN)  # Sharpen the image
    return image

# Function to extract max and min values for stress images
def extract_stress_values(image_path):
    image = preprocess_image(image_path)
    text = pytesseract.image_to_string(image)

    max_value = None
    min_value = None

    for line in text.splitlines():
        if "Max" in line:
            max_match = re.search(r"(\d+\.\d+|\d+)", line)
            if max_match:
                max_value = float(max_match.group(1))
        if "Min" in line:
            min_match = re.search(r"(\d+\.\d+|\d+)", line)
            if min_match:
                min_value = float(min_match.group(1))
    
    # Correct misread large values
    if max_value and max_value > 1000:
        max_value /= 1000
    if min_value and min_value > 1000:
        min_value /= 1000
    
    return max_value, min_value

# Function to extract only max deformation value
def extract_deformation_value(image_path):
    image = preprocess_image(image_path)
    text = pytesseract.image_to_string(image)

    max_value = None

    for line in text.splitlines():
        if "Max" in line:
            max_match = re.search(r"(\d+\.\d+|\d+)", line)
            if max_match:
                max_value = float(max_match.group(1))
                break  # Stop after finding the first valid max value
    
    # Correct misread large values
    if max_value and max_value > 1000:
        max_value /= 1000
    
    return max_value


# Function to generate captions
def generate_caption(max_value, min_value=None, is_stress=True):
    if is_stress:
        if max_value is not None and min_value is not None:
            return f"The max stress is {max_value} and the min stress is {min_value}."
        return "Could not extract values from the image."
    else:
        if max_value is not None:
            return f"The max deformation is {max_value}."
        return "Could not extract values from the image."

# Function to create a PowerPoint report
def create_ppt_report(image_path, caption, output_file="report_s.pptx"):
    prs = Presentation()
    slide = prs.slides.add_slide(prs.slide_layouts[5])  # Blank slide layout

    # Add image to slide
    slide.shapes.add_picture(image_path, Inches(1), Inches(1.5), height=Inches(3))

    # Add caption below the image
    textbox = slide.shapes.add_textbox(Inches(1), Inches(4.8), Inches(8), Inches(1))
    text_frame = textbox.text_frame
    text_frame.text = caption

    # Save the presentation
    prs.save(output_file)
    print(f"PowerPoint report saved as {output_file}")

# Paths to the uploaded image
image_path = "D:\\BTech_Project\\demo_code\\demo_code\\UPDATED\\STRESS\\3.png"  # Replace with the correct file path

# Determine if the image is stress or deformation based on logic
is_stress_image = True  # Set True for stress images, False for deformation images

# Process based on image type
if is_stress_image:
    max_val, min_val = extract_stress_values(image_path)
    caption = generate_caption(max_val, min_val, is_stress=True)
else:
    max_val = extract_deformation_value(image_path)
    caption = generate_caption(max_val, is_stress=False)

# Create PowerPoint report
create_ppt_report(image_path, caption)


PowerPoint report saved as report_s.pptx


In [14]:
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter
import re
from pptx import Presentation
from pptx.util import Inches

# Path to Tesseract OCR binary (adjust the path for your system)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Function to preprocess the image for better OCR
def preprocess_image(image_path):
    image = Image.open(image_path)
    image = image.convert("L")  # Convert to grayscale
    image = ImageEnhance.Contrast(image).enhance(5)  # Further increase contrast
    image = image.filter(ImageFilter.SHARPEN)  # Sharpen the image
    return image

# Function to extract max deformation value
def extract_deformation_value(image_path):
    image = preprocess_image(image_path)
    
    # Use Tesseract's configuration for page segmentation and whitelisting relevant characters
    config = '--psm 6 -c tessedit_char_whitelist=0123456789.Max'  # Allow only digits and 'Max'
    text = pytesseract.image_to_string(image, config=config)

    max_value = None

    print("OCR Text for Deformation Extraction:", text)  # Debug: Print OCR output for review

    # Try to find the "Max" keyword and extract the corresponding numerical value
    for line in text.splitlines():
        # Use regex to find "Max" followed by a valid number
        if "Max" in line:
            max_match = re.search(r"(\d+\.\d+|\d+)", line)
            if max_match:
                max_value = float(max_match.group(1))
                break  # Stop after finding the first valid max value
    
    # If max_value is read but appears too large (such as due to OCR errors), correct it
    if max_value and max_value > 1000:
        max_value /= 1000
    
    return max_value

# Function to generate captions
def generate_caption(max_value, is_stress=False):
    if is_stress:
        return f"The max stress is {max_value}."
    else:
        if max_value is not None:
            return f"The max deformation is {max_value}."
        return "Could not extract values from the image."

# Function to create a PowerPoint report
def create_ppt_report(image_path, caption, output_file="report_d.pptx"):
    prs = Presentation()
    slide = prs.slides.add_slide(prs.slide_layouts[5])  # Blank slide layout

    # Add image to slide
    slide.shapes.add_picture(image_path, Inches(1), Inches(1.5), height=Inches(3))

    # Add caption below the image
    textbox = slide.shapes.add_textbox(Inches(1), Inches(4.8), Inches(8), Inches(1))
    text_frame = textbox.text_frame
    text_frame.text = caption

    # Save the presentation
    prs.save(output_file)
    print(f"PowerPoint report saved as {output_file}")

# Path to the uploaded image
image_path = "D:\\BTech_Project\\demo_code\\demo_code\\UPDATED\\DEFORMATION\\5.png"  # Replace with the correct file path

# Process deformation image (set to False for deformation images)
is_stress_image = False  # Set False for deformation images

# Process based on image type
if is_stress_image:
    max_val, min_val = extract_stress_values(image_path)
    caption = generate_caption(max_val, min_val, is_stress=True)
else:
    max_val = extract_deformation_value(image_path)
    caption = generate_caption(max_val, is_stress=False)

# Create PowerPoint report
create_ppt_report(image_path, caption)


OCR Text for Deformation Extraction: 20242
0.15823Max
a
a
a
a ax
5

PowerPoint report saved as report_d.pptx
